perform data processing for oot sample

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from scipy.stats import entropy
import scorecardpy as sc
%matplotlib inline 

In [3]:
full = pd.read_pickle('./data/010_full.pkl')
oot = full[full.monthlyReportingPeriod == 202406]

#### missing values

In [3]:
# missing values impute using median
oot['creditScore'] = oot['creditScore'].replace(to_replace=9999, value=np.nan)
oot = oot.fillna(value={'creditScore': oot['creditScore'].median()})

oot['originalDebtToIncomeRatio'] = oot['originalDebtToIncomeRatio'].replace(to_replace=999, value=np.nan)
oot = oot.fillna(value={'originalDebtToIncomeRatio': oot['originalDebtToIncomeRatio'].median()})

# due to high mutual information value w/ postalCode
oot = oot.drop(columns='areaCode') 

# due to 0.98 correlation with originalLoanToValue
oot = oot.drop(columns='originalCombinedLoanToValue') 

# not useful
oot = oot.drop(columns=['sellerName', 'servicerName'])

# superConformingFlag
# according to the user guide, nulls are actully Not Super Conforming
oot['superConformingFlag'] = oot['superConformingFlag'].fillna('N')

# this is a seq number which has no meaning & 98% are missing, so drop it
oot.drop(columns='preReliefRefinanceLoanSeqNumber', inplace=True)

# nulls are actually 'N', as opposed to 'Y' for this feature
oot['reliefRefinanceIndicator'] = oot['reliefRefinanceIndicator'].fillna('N') 

# miscallenous
# '9' means Not available or not applicable, replace for readability
oot['programIndicator'] = oot['programIndicator'].replace(to_replace='9', value='NA')
oot['propertyValuationMethod'] = oot['propertyValuationMethod'].replace(9, 'NA')

/var/folders/33/qv3yf4892xld1btchk4myrjc0000gn/T/ipykernel_24905/3744530987.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oot['creditScore'] = oot['creditScore'].replace(to_replace=9999, value=np.nan)


In [4]:
oot = oot.drop(columns=['zeroBalanceCode', 'zeroBalanceEffectiveDate']) 

# nulls are a category by themselves, thus replace with values.
oot['modificationFlag'] = oot['modificationFlag'].fillna('N')

oot['stepModificationFlag'] = oot['stepModificationFlag'].fillna('NotModified')

oot['paymentDeferral'] = oot['paymentDeferral'].fillna('N')

oot['borrowerAssistanceStatusCode'] = oot['borrowerAssistanceStatusCode'].fillna('N')

oot['delinquencyDueToDisaster'] = oot['delinquencyDueToDisaster'].fillna('N')

oot = oot.drop(columns=['defectSettlementDate','miRecoveries',
                          'netSaleProceeds', 'nonMiRecoveries', 'actualLossCalculation',
                          'zeroBalanceRemovalUpb', 'delinquentAccruedInterest'])

oot = oot.drop(columns=['totalExpenses','legalCosts',
                          'maintenanceAndPreservationCosts', 'taxesAndInsurance',
                          'miscellaneousExpenses'])

idx = oot[oot['cumulativeModificationCost'].notnull()].index[3]
oot.loc[idx-6: idx+4, ['loanSequenceNumber', 'monthlyReportingPeriod',
                         'modificationFlag', 'paymentDeferral',
                         'cumulativeModificationCost', 'currentMonthModificationCost']]

,loanSequenceNumber,monthlyReportingPeriod,modificationFlag,paymentDeferral,cumulativeModificationCost,currentMonthModificationCost
353932,F18Q20040841,202406,N,P,466.58,11.38


In [5]:
oot = oot.drop(columns=['cumulativeModificationCost'])
oot['currentMonthModificationCost'] = oot['currentMonthModificationCost'].fillna(0)

In [6]:
# this variabe's in formation is caputured by maturityDate, so drop
oot = oot.drop(columns=['dueDateOfLastPaidInstallment'])

# estimatedLoanToValue 999 means Unknown
# if currentActualUpb==0, then it is likely Unknown
# not vice versa
print(sum(oot.estimatedLoanToValue == 999))
print(sum(oot.currentActualUpb == 0))
print(sum((oot.currentActualUpb == 0) & (oot.estimatedLoanToValue == 999)))

oot['estimatedLoanToValue'] = oot['estimatedLoanToValue'].replace(to_replace=999, value=np.nan)
oot = oot.fillna(value={'estimatedLoanToValue': oot['estimatedLoanToValue'].median()})

# clean the variables just created
oot['upbPctChange'] = oot['upbPctChange'].fillna(0)

oot.isnull().sum()

204
257
204


loanSequenceNumber                0
monthlyReportingPeriod            0
currentActualUpb                  0
currentLoanDelinquencyStatus      0
loanAge                           0
remainingMonthsToLegalMaturity    0
modificationFlag                  0
currentInterestRate               0
currentNonInterestBearingUpb      0
stepModificationFlag              0
paymentDeferral                   0
estimatedLoanToValue              0
delinquencyDueToDisaster          0
borrowerAssistanceStatusCode      0
currentMonthModificationCost      0
interestBearingUpb                0
target                            0
upbPctChange                      0
nonPmts_3m                        0
delinquencyDueToDisaster_hist     0
interestBearingUpb_ratio          0
creditScore                       0
firstPaymentDate                  0
firstTimeHomebuyerFlag            0
maturityDate                      0
mortgageInsurancePct              0
numberOfUnits                     0
occupancyStatus             

#### removing outliers - not run

In [ ]:
# save the cleaned dataset
oot.to_pickle("./data/012_oot_cleaned.pkl")
oot.to_csv("./data/012_oot_cleaned.csv")

#### apply the existing WoE binning to the test set

In [ ]:
oot = pd.read_pickle("./data/012_oot_cleaned.pkl")
oot.isnull().any()

loanSequenceNumber                False
monthlyReportingPeriod            False
currentActualUpb                  False
currentLoanDelinquencyStatus      False
loanAge                           False
remainingMonthsToLegalMaturity    False
modificationFlag                  False
currentInterestRate               False
currentNonInterestBearingUpb      False
stepModificationFlag              False
paymentDeferral                   False
estimatedLoanToValue              False
delinquencyDueToDisaster          False
borrowerAssistanceStatusCode      False
currentMonthModificationCost      False
interestBearingUpb                False
target                            False
upbPctChange                      False
nonPmts_3m                        False
delinquencyDueToDisaster_hist     False
interestBearingUpb_ratio          False
creditScore                       False
firstPaymentDate                  False
firstTimeHomebuyerFlag            False
maturityDate                      False


In [9]:
oot = oot.drop(columns=['loanSequenceNumber', 'monthlyReportingPeriod'])

# for binning to work well:
oot['target'] = oot['target'].astype(int)

# Convert object (string) and category type columns to string
for col in oot.select_dtypes(include=['object', 'category']).columns:
    oot[col] = oot[col].astype(str)

# these are below 0.02
vars = [
    'delinquencyDueToDisaster_hist', 
    'superConformingFlag',
    'programIndicator',
    'channel',
    'currentMonthModificationCost',
    'interestBearingUpb_ratio',
    'reliefRefinanceIndicator',
    'paymentDeferral',
    'stepModificationFlag',
    'modificationFlag',
    'numberOfUnits',
    'borrowerAssistanceStatusCode',
    'currentNonInterestBearingUpb',
    'delinquencyDueToDisaster',
    'currentLoanDelinquencyStatus'
    ]
oot = oot.drop(columns=vars)

# these only have 1 unique value
vars= [
    'prepaymentPenaltyMortgageFlag',
    'amortizationType',
    'interestOnlyIndicator'
]
oot = oot.drop(columns=vars)

In [10]:
import pickle
with open('bins.pkl', 'rb') as file:
    bins = pickle.load(file)

oot_woe = sc.woebin_ply(oot, bins)
oot_woe.head()

[INFO] converting into woe values ...


,target,occupancyStatus_woe,mortgageInsurancePct_woe,currentInterestRate_woe,firstTimeHomebuyerFlag_woe,postalCode_woe,nonPmts_3m_woe,creditScore_woe,originalLoanTerm_woe,numberOfBorrowers_woe,...,originalDebtToIncomeRatio_woe,loanPurpose_woe,remainingMonthsToLegalMaturity_woe,firstPaymentDate_woe,propertyState_woe,interestBearingUpb_woe,originalUpb_woe,propertyType_woe,estimatedLoanToValue_woe,originalInterestRate_woe
1908,1,0.050082,-0.357003,1.136875,-0.230913,0.609031,2.577330,1.049741,0.145830,0.371297,...,0.292155,-0.480342,-1.051984,0.914414,0.609031,0.654611,-0.293348,0.075908,0.002212,1.169776
2043,1,0.050082,1.076423,1.136875,-0.230913,0.609031,2.577330,1.049741,0.145830,0.371297,...,0.292155,-0.480342,-1.051984,0.914414,0.609031,0.654611,0.453772,0.075908,0.377895,1.169776
11980,0,-0.518279,-0.357003,0.373073,-0.230913,0.609031,-1.474355,-1.274056,-0.732154,0.371297,...,-0.335823,0.404697,-0.140859,0.914414,0.609031,0.654611,1.040642,0.075908,0.002212,0.341579
18494,0,0.050082,-0.357003,0.373073,-0.230913,0.609031,-1.474355,0.243703,-0.732154,-0.720390,...,-0.335823,-0.133258,-0.140859,0.914414,0.609031,0.654611,0.453772,0.075908,0.002212,0.341579
20877,0,0.050082,-0.357003,1.136875,-0.230913,-0.143293,-1.474355,1.049741,-0.380230,0.371297,...,0.292155,-0.480342,-0.140859,0.914414,-0.143293,0.654611,0.115428,0.075908,0.002212,1.169776


In [ ]:
oot_woe = oot_woe.drop(columns=['interestBearingUpb_woe',
                        'postalCode_woe', 'originalInterestRate_woe'])

oot_woe.to_pickle("./data/012_oot_woe.pkl")
oot_woe.columns

Index(['target', 'occupancyStatus_woe', 'mortgageInsurancePct_woe',
       'currentInterestRate_woe', 'firstTimeHomebuyerFlag_woe',
       'nonPmts_3m_woe', 'creditScore_woe', 'originalLoanTerm_woe',
       'numberOfBorrowers_woe', 'originalLoanToValue_woe',
       'currentActualUpb_woe', 'upbPctChange_woe', 'maturityDate_woe',
       'loanAge_woe', 'propertyValuationMethod_woe',
       'miCancellationIndicator_woe', 'originalDebtToIncomeRatio_woe',
       'loanPurpose_woe', 'remainingMonthsToLegalMaturity_woe',
       'firstPaymentDate_woe', 'propertyState_woe', 'originalUpb_woe',
       'propertyType_woe', 'estimatedLoanToValue_woe'],
      dtype='object')